### Notes

For Span obj:
+ build off author and publisher names
+ text
+ start char end char
+ label (author or publisher name)
+ group (make sure authors and pubs don't overlap)
+ as a group (this group's closeness to KB ids)
    + dictionary of id and then its score
    
For ReviewObj:
+ all author spans attribute
+ all pub spans attribute

Also refer to spaCy spans -> https://spacy.io/api/span
+ doc (doc object)
+ start (int)
+ end (int)
+ label (int/unicode)
+ kb_id (int/unicode)
+ vector (numpy.ndarray[ndim=1, dtype='float32'])

In [1]:
import sys
sys.path.append('../')

In [2]:
from application.name_obj_classes import PubName, PersonName
from application.review_obj_class import ReviewObj
import pandas as pd
import os
%pprint

Pretty printing has been turned OFF


In [3]:
from nltk.metrics import edit_distance

In [4]:
# loading in files
# directory = "../../aps_reviews_50/aps_reviews/"
directory = "../../aps_reviews_1000/"
filenames = os.listdir(directory)

In [5]:
# reviews = ((file.split('.')[0], open(directory + file).read()) for file in filenames)
reviews = [(file.split('.')[0], open(directory + file).read()) for file in filenames]

In [6]:
problem_texts = []
for rev in reviews:
    try:
        ReviewObj(*rev)
    except:
        problem_texts.append(rev)

In [7]:
len(problem_texts)

10

In [8]:
ReviewObj(*problem_texts[0])

KeyError: 'directory-on'

In [ ]:
for 
review_list = [ReviewObj(file, txt) for (file, txt) in reviews]

In [ ]:
review_list[0].cleaned_text

Span object will contain:
+ Review ID (parent doc)
+ Start char (int)
+ End char (int)
+ Label (person/publisher)
+ Group (int)
+ Name variations
+ VIAF but not yet

In [ ]:
# review id
review_list[0].person_names[0].review_id

In [ ]:
# start and end char
review_list[0].person_names[0].review_loc_chars

In [ ]:
# label
review_list[0].person_names[0].name_type

In [ ]:
review_list[0].person_names[0].getNameVariants()

In [ ]:
class NameSpan():
    def __init__(self, NameObj):
        self.name = NameObj
        self.review_id = NameObj.review_id
        self.review_doc = '' # ReviewObj will go here?
        self.span = NameObj.review_loc_chars
        self.start_char = NameObj.review_loc_chars[0]
        self.end_char = NameObj.review_loc_chars[1]
        self.label = NameObj.name_type
        self.name_id = int(span_test.review_id + str(self.start_char))
        self.group = -1

In [ ]:
span_test = NameSpan(review_list[0].person_names[0])

In [ ]:
span_test.review_id

In [ ]:
span_test.name_id

In [ ]:
span_test.name

In [ ]:
span_test.name.last_name

In [ ]:
spanlist = [NameSpan(x) for x in review_list[0].person_names]

In [ ]:
spanlist

In [ ]:
names = sorted([(span.name, span.name_id) for span in spanlist], key=lambda x: len(x[0].last_name))

In [ ]:
names

In [ ]:
names[0].last_name

In [ ]:
group_dict = {}
used_ids = []

In [ ]:
for e, (name, name_id) in enumerate(names):
    if (name_id not in used_ids):
        id_holder = [name_id]
        for name2, name_id2 in names:
            if (e < (len(name) - 1)) and (name_id!=name_id2):
                if (edit_distance(name.last_name, name2.last_name[:len(name.last_name)+1]) < 2) and (name_id2 not in used_ids):
                    if (name.first_initial==name2.first_initial and name.middle_initial==name2.middle_initial or name.title==name2.title) or (name.first_initial==name2.first_initial or name.middle_initial==name2.middle_initial and name.title==name2.title):
                        id_holder.append(name_id2)
                        used_ids.append(name_id2)
        used_ids.append(name_id)
        for x in id_holder:
            group_dict[x] = e

In [ ]:
group_dict

In [ ]:
for span in spanlist:
    span.group = group_dict[span.name_id]

In [ ]:
for span in spanlist:
    print(span.name, span.group)

In [ ]:
def group_people(spanlist):
    
    names = sorted([(span.name, span.name_id) for span in spanlist], key=lambda x: len(x[0].last_name))
    group_dict = {}
    used_ids = []
    for e, (name, name_id) in enumerate(names):
        if (name_id not in used_ids):
            id_holder = [name_id]
            for name2, name_id2 in names:
                if (e < (len(name) - 1)) and (name_id!=name_id2):
                    if (edit_distance(name.last_name, name2.last_name[:len(name.last_name)+1]) < 2) and (name_id2 not in used_ids):
                        if (name.first_initial==name2.first_initial and name.middle_initial==name2.middle_initial or name.title==name2.title) or (name.first_initial==name2.first_initial or name.middle_initial==name2.middle_initial and name.title==name2.title):
                            id_holder.append(name_id2)
                            used_ids.append(name_id2)
            used_ids.append(name_id)
            for x in id_holder:
                group_dict[x] = e
    
    for span in spanlist:
        span.group = group_dict[span.name_id]
    
    return spanlist

In [ ]:
spanlist = [NameSpan(x) for x in review_list[0].person_names]

In [ ]:
spanlist = group_people(spanlist)

In [ ]:
for span in spanlist:
    print(span.name, span.group)

+ change names to have underscores for tokenization
+ sentence tokenize first?

In [ ]:
class NameSpanGenerator():
    def _get_group:
        
    def _get_group_person